# [Multi Agent Chat](https://microsoft.github.io/autogen/docs/Examples/#automated-multi-agent-chat)

https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat/

## (1) [Automated Task Solving by Group Chat (with 3 group member agents and 1 manager agent)](https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat/)

In [4]:
! pip install pyautogen -U

  Using cached pyautogen-0.2.32-py3-none-any.whl.metadata (27 kB)
Using cached pyautogen-0.2.32-py3-none-any.whl (314 kB)
  Attempting uninstall: pyautogen
    Found existing installation: pyautogen 0.2.31
    Uninstalling pyautogen-0.2.31:
      Successfully uninstalled pyautogen-0.2.31


In [8]:
import autogen

In [9]:
# Set your API endpoints
config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
)

In [10]:
# Set up local command code executor
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always/Never take human input for this agent for safety.
)

In [11]:
# Construct Agents
llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)

coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [12]:
# Start chat

user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
# type exit to terminate the chat



User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

**Plan**

1. **Search arXiv**: Use a Python script to search for recent papers about "GPT-4" on arXiv.
2. **Collect paper information**: Parse the response to find relevant details about the latest paper.
3. **Identify potential applications**: Read the abstract of the paper to identify potential applications of GPT-4 in software.

**Step 1: Search arXiv**  
I'll use the arXiv API to search for the latest papers on GPT-4. 

```python
# filename: search_gpt4_arxiv.py
import requests

# Define the search query and sort by submission date in descending order
search_query = "

ChatResult(chat_id=None, chat_history=[{'content': 'Find a latest paper about gpt-4 on arxiv and find its potential applications in software.', 'role': 'assistant'}, {'content': '', 'role': 'assistant'}, {'content': '**Plan**\n\n1. **Search arXiv**: Use a Python script to search for recent papers about "GPT-4" on arXiv.\n2. **Collect paper information**: Parse the response to find relevant details about the latest paper.\n3. **Identify potential applications**: Read the abstract of the paper to identify potential applications of GPT-4 in software.\n\n**Step 1: Search arXiv**  \nI\'ll use the arXiv API to search for the latest papers on GPT-4. \n\n```python\n# filename: search_gpt4_arxiv.py\nimport requests\n\n# Define the search query and sort by submission date in descending order\nsearch_query = "gpt-4"\nurl = f"http://export.arxiv.org/api/query?search_query=all:{search_query}&sortBy=submittedDate&sortOrder=descending&max_results=1"\n\nresponse = requests.get(url)\n\n# Print the resp

## (2) [Automated Complex Task Solving by Group Chat (with 6 group member agents and 1 manager agent)](https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat_research/)

In [13]:
import autogen


# Set your API endpoints
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o","gpt-4-turbo-1106","gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)



In [14]:
# Construct Agents

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
    human_input_mode="TERMINATE",
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=gpt4_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [16]:
# Start the chat
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan

1. **Step 1: Data Collection**
   - **Scientist**: Identify the relevant keywords and search criteria for finding papers on LLM (Large Language Models) applications.
   - **Engineer**: Write a script to scrape the arXiv API for papers published in the last week using the identified keywords.

2. **Step 2: Data Filtering**
   - **Engineer**: Filter the scraped data to extract papers specifically related to LLM applications.
   - **Scientist**: Review the filtered list to ensure relevance and accuracy.

3. **Step 3: Domain Classification**
   - **Scientist**: Classify the papers into different domains based on their abstracts and titles.

4. **Step 4: Markdown Table Creation**
   - **Engineer**: Write a script to 

ChatResult(chat_id=None, chat_history=[{'content': '\nfind papers on LLM applications from arxiv in the last week, create a markdown table of different domains.\n', 'role': 'assistant'}, {'content': '### Plan\n\n1. **Step 1: Data Collection**\n   - **Scientist**: Identify the relevant keywords and search criteria for finding papers on LLM (Large Language Models) applications.\n   - **Engineer**: Write a script to scrape the arXiv API for papers published in the last week using the identified keywords.\n\n2. **Step 2: Data Filtering**\n   - **Engineer**: Filter the scraped data to extract papers specifically related to LLM applications.\n   - **Scientist**: Review the filtered list to ensure relevance and accuracy.\n\n3. **Step 3: Domain Classification**\n   - **Scientist**: Classify the papers into different domains based on their abstracts and titles.\n\n4. **Step 4: Markdown Table Creation**\n   - **Engineer**: Write a script to generate a markdown table from the classified data.\n\n

In [12]:
# Group Chat WITHOUT Critic for Comparison
groupchat_nocritic = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=50
)
for agent in groupchat.agents:
    agent.reset()
manager_nocritic = autogen.GroupChatManager(groupchat=groupchat_nocritic, llm_config=gpt4_config)
user_proxy.initiate_chat(
    manager_nocritic,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan

1. **Step 1: Data Collection**
   - **Scientist**: Identify the relevant keywords and search criteria for finding papers on LLM (Large Language Models) applications.
   - **Engineer**: Write a script to scrape the arXiv API for papers published in the last week using the identified keywords.

2. **Step 2: Data Filtering**
   - **Engineer**: Filter the scraped data to extract papers specifically related to LLM applications.
   - **Scientist**: Review the filtered list to ensure relevance and accuracy.

3. **Step 3: Domain Classification**
   - **Scientist**: Classify the papers into different domains based on their abstracts and titles.

4. **Step 4: Markdown Table Creation**
   - **Engineer**: Write a script to 

ChatResult(chat_id=None, chat_history=[{'content': '\nfind papers on LLM applications from arxiv in the last week, create a markdown table of different domains.\n', 'role': 'assistant'}, {'content': '### Plan\n\n1. **Step 1: Data Collection**\n   - **Scientist**: Identify the relevant keywords and search criteria for finding papers on LLM (Large Language Models) applications.\n   - **Engineer**: Write a script to scrape the arXiv API for papers published in the last week using the identified keywords.\n\n2. **Step 2: Data Filtering**\n   - **Engineer**: Filter the scraped data to extract papers specifically related to LLM applications.\n   - **Scientist**: Review the filtered list to ensure relevance and accuracy.\n\n3. **Step 3: Domain Classification**\n   - **Scientist**: Classify the papers into different domains based on their abstracts and titles.\n\n4. **Step 4: Markdown Table Creation**\n   - **Engineer**: Write a script to generate a markdown table from the classified data.\n\n